In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import torch
import pdb
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb

from dataset import load_data
from matrix_factorization import MF, MF_CVIB, MF_IPS, MF_SNIPS, MF_DR
from utils import gini_index, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)

dataset_name = "coat"

In [2]:
if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)

===>Load from coat data set<===
[train] rating ratio: 0.080000
[test]  rating ratio: 0.053333
# user: 290, # item: 300


In [3]:
"MF CVIB"
mf_cvib = MF_CVIB(num_user, num_item)
mf_cvib.fit(x_train, y_train, 
    lr=0.01,
    batch_size=128,
    lamb=1e-4,
    alpha=0.1,
    tol=1e-5,
    verbose=False)

test_pred = mf_cvib.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
print("***"*5 + "[MF-CVIB]" + "***"*5)
print("[MF-CVIB] test mse:", mse_func(y_test, test_pred))
print("[MF-CVIB] test auc:", auc_mf)
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-CVIB]" + "***"*5)

[MF-CVIB] epoch:63, xent:19.044715628027916
***************[MF-CVIB]***************
[MF-CVIB] test mse: 0.21890791943751317
[MF-CVIB] test auc: 0.7211649920852734
Num User: 290
Gini index: 0.30810522573764754
Global utility: 0.5351724137931034
***************[MF-CVIB]***************


In [4]:
"MF naive"
mf = MF(num_user, num_item)
mf.fit(x_train, y_train, lr=0.01, lamb=1e-4,verbose=0)
test_pred = mf.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
print("***"*5 + "[MF]" + "***"*5)
print("[MF] test mse:", mse_func(y_test, test_pred))
print("[MF] test auc:", auc_mf)
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF]" + "***"*5)

[MF] epoch:85, xent:17.313761726021767
***************[MF]***************
[MF] test mse: 0.24509646945574923
[MF] test auc: 0.7020252343292666
Num User: 290
Gini index: 0.34480347238968034
Global utility: 0.49310344827586206
***************[MF]***************


In [5]:
"MF IPS"
mf_ips = MF_IPS(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_ips.fit(x_train, y_train, y_ips=y_ips,lr=0.05, lamb=1e-3,verbose=0)
test_pred = mf_ips.predict(x_test)
mse_mfips = mse_func(y_test, test_pred)
auc_mfips = roc_auc_score(y_test, test_pred)
print("***"*5 + "[MF-IPS]" + "***"*5)
print("[MF-IPS] test mse:", mse_func(y_test, test_pred))
print("[MF-IPS] test auc:", auc_mfips)
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-IPS]" + "***"*5)

[MF-IPS] epoch:25, xent:217.241384267807
***************[MF-IPS]***************
[MF-IPS] test mse: 0.22992293407446174
[MF-IPS] test auc: 0.7156132772536092
Num User: 290
Gini index: 0.3265323102523816
Global utility: 0.5144827586206896
***************[MF-IPS]***************


In [6]:
"MF-SNIPS"
mf_snips = MF_SNIPS(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_snips.fit(x_train, y_train, y_ips=y_ips,lr=0.05, lamb=1e-4,verbose=0)
test_pred = mf_snips.predict(x_test)
mse_mfsnips = mse_func(y_test, test_pred)
auc_mfsnips = roc_auc_score(y_test, test_pred)
print("***"*5 + "[MF-SNIPS]" + "***"*5)
print("[MF-SNIPS] test mse:", mse_mfsnips)
print("[MF-SNIPS] test auc:", auc_mfsnips)
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-SNIPS]" + "***"*5)

[MF-SNIPS] epoch:26, xent:18.89180314540863
***************[MF-SNIPS]***************
[MF-SNIPS] test mse: 0.23741498000287292
[MF-SNIPS] test auc: 0.6960321584584727
Num User: 290
Gini index: 0.3413887190102094
Global utility: 0.5055172413793104
***************[MF-SNIPS]***************


In [7]:
"MF DR"
mf_dr = MF_DR(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_dr.fit(x_train, y_train, y_ips=y_ips,lr=0.05, lamb=1e-3,verbose=0)
test_pred = mf_dr.predict(x_test)
mse_mfdr = mse_func(y_test, test_pred)
auc_mfdr = roc_auc_score(y_test, test_pred)
print("***"*5 + "[MF-DR]" + "***"*5)
print("[MF-DR] test mse:", mse_mfdr)
print("[MF-DR] test auc:", auc_mfdr)
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-DR]" + "***"*5)

[MF-DR] epoch:29, xent:28659.92889404297
***************[MF-DR]***************
[MF-DR] test mse: 0.23569433182636876
[MF-DR] test auc: 0.7058138249047488
Num User: 290
Gini index: 0.33353853274705036
Global utility: 0.5137931034482759
***************[MF-DR]***************
